# Estimación de dirección de arribo de fuentes sonoras 

In [1]:
import numpy as np
import pandas as pd
import gspread # Extension para leer el excel desde Drive
from oauth2client.service_account import ServiceAccountCredentials # Extensión para autentificación con Drive
from tdoa_py import estimation, simulation_room

In [2]:
# Configurar credenciales
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("simulationsdoa-credenciales.json", scope)
client = gspread.authorize(creds)

# Abrir la hoja de cálculo
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/13XTDng98P99pfexK78Dd4Gud1CzZwO7PfVhpyIG1jCM/edit?gid=0#gid=0")
sheet = spreadsheet.sheet1

# Leer datos
data = sheet.get_all_records()
df_simulations = pd.DataFrame(data)

df_simulations["room_dim"] = df_simulations.apply(lambda row: (row["room_x"], row["room_y"], row["room_z"]), axis=1) # Para pasar el recinto en una tupla luego
df_simulations.head() # Verificar el resultado

,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,theta_3,t_12,t_13,t_14,avg_angle_deg,avg_tdoa,hemi_1,hemi_2,room_dim,error
0,1,phat,audio_anecoico_corto.wav,50,100,8,3,70,4,0.1,...,12.42,0.000271,0.000563,0.000854,16.47,0.000563,,,"(50, 100, 8)",7.24
1,1,phat,audio_anecoico_corto.wav,50,100,8,3,70,4,0.1,...,12.42,0.000271,0.000563,0.000854,16.47,0.000563,,,"(50, 100, 8)",2.24
2,1,phat,audio_anecoico_corto.wav,50,100,8,3,70,4,0.1,...,17.68,0.000271,0.000563,0.000833,18.23,0.000556,,,"(50, 100, 8)",-2.76
3,1,phat,audio_anecoico_corto.wav,50,100,8,3,70,4,0.1,...,42.40,0.000250,0.000542,0.000646,31.70,0.000479,,,"(50, 100, 8)",-7.76
4,1,phat,audio_anecoico_corto.wav,50,100,8,3,70,4,0.1,...,44.39,0.000250,0.000521,0.000625,34.02,0.000465,,,"(50, 100, 8)",-9.68


In [3]:
# Iterar sobre el DataFrame con indexación correcta
for idx, sim in df_simulations.iterrows():
    src_az_deg, mic_signals = simulation_room.sim_room_Nmics(
        wav_path=f'audios/anechoic/{sim["audio"]}',
        out_dir=f'audios/output/{sim["sim_name"]}',
        fs=48000,
        room_dim=sim["room_dim"],
        rt60=sim["rt60"],
        snr_db=sim["snr_db"],
        n_mics=sim["n_mics"],
        mic_d=sim["mic_d"],
        mic_z=sim["mic_z"],
        mic_directivity=sim["mic_directivity"],
        src_dist=sim["src_dist"],
        src_az_deg=sim["src_az_deg"],
        src_z=sim["src_z"],
        save_audio=False
    )
    
    avg_angle, avg_tdoa, angles, tdoas = estimation.estimate_doa(
        signals=mic_signals,
        fs=48000,
        d=sim["mic_d"],
        method=sim["method"]
    )

    # Guardar resultados en cada fila individual
    # print(angles)
    # print(tdoas)
    df_simulations.at[idx, 'avg_angle_deg'] = avg_angle
    df_simulations.at[idx, 'avg_tdoa'] = avg_tdoa
    df_simulations.at[idx, 'theta_1'] = angles[0]
    df_simulations.at[idx, 'theta_2'] = angles[1]
    df_simulations.at[idx, 'theta_3'] = angles[2]
    df_simulations.at[idx, 't_12'] = tdoas[0]
    df_simulations.at[idx, 't_13'] = tdoas[1]
    df_simulations.at[idx, 't_14'] = tdoas[2]
    # print("phi_deg", phi_deg)
    # print("tdoas", tdoas)

# # Convertir el DataFrame a una lista de listas para Google Sheets
df_simulations["room_dim"] = df_simulations["room_dim"].apply(lambda x: ",".join(map(str, x)))
data = [df_simulations.columns.values.tolist()] + df_simulations.values.tolist()

# Escribir en la hoja de cálculo
sheet.update(data)
print("DataFrame guardado correctamente en Google Sheets.")
df_simulations



DataFrame guardado correctamente en Google Sheets.


,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,theta_3,t_12,t_13,t_14,avg_angle_deg,avg_tdoa,hemi_1,hemi_2,room_dim,error
0,1,phat,audio_anecoico_corto.wav,50,100,8,3,70,4,0.1,...,12.420011,0.000271,0.000562,0.000854,16.472908,0.000562,,,"50,100,8",7.24
1,1,phat,audio_anecoico_corto.wav,50,100,8,3,70,4,0.1,...,12.420011,0.000271,0.000562,0.000854,16.472908,0.000562,,,"50,100,8",2.24
2,1,phat,audio_anecoico_corto.wav,50,100,8,3,70,4,0.1,...,17.678070,0.000271,0.000562,0.000833,18.225595,0.000556,,,"50,100,8",-2.76
3,1,phat,audio_anecoico_corto.wav,50,100,8,3,70,4,0.1,...,42.404466,0.000250,0.000542,0.000646,31.698140,0.000479,,,"50,100,8",-7.76
4,1,phat,audio_anecoico_corto.wav,50,100,8,3,70,4,0.1,...,44.390938,0.000250,0.000521,0.000625,34.024012,0.000465,,,"50,100,8",-9.68
5,1,phat,audio_anecoico_corto.wav,50,100,8,3,70,4,0.1,...,28.198061,0.000188,0.000500,0.000771,36.378597,0.000486,,,"50,100,8",-14.68
6,1,phat,audio_anecoico_corto.wav,50,100,8,3,70,4,0.1,...,49.974760,0.000229,0.000479,0.000562,40.965061,0.000424,,,"50,100,8",-17.27
7,1,phat,audio_anecoico_corto.wav,50,100,8,3,70,4,0.1,...,38.182877,0.000167,0.000458,0.000687,43.833075,0.000437,,,"50,100,8",-20.20
8,1,phat,audio_anecoico_corto.wav,50,100,8,3,70,4,0.1,...,42.404466,0.000208,0.000333,0.000646,47.309625,0.000396,,,"50,100,8",-25.20
9,1,phat,audio_anecoico_corto.wav,50,100,8,3,70,4,0.1,...,58.397183,0.000188,0.000396,0.000458,51.872592,0.000347,,,"50,100,8",-28.34
